In [1]:
import pandas as pd
from bs4 import BeautifulSoup as soup
import csv
from pathlib import Path

import datetime

now = datetime.datetime.now()

In [2]:
import selenium
from selenium import webdriver

In [3]:
# konfigurasi chromedriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--window-size=1420,1080')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')

browser = webdriver.Chrome(chrome_options=chrome_options)

/opt/tljh/user/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


In [4]:

data_folder = Path("data/")
file_data = 'data_kamar_isolasi_detail-' + now.strftime("%Y%m%d") + '-' + now.strftime("%H%M") + '.csv'
filename = data_folder / file_data
csvFile = open(filename, 'a')
#Use csv Writer
csvWriter = csv.writer(csvFile)
csvWriter.writerow(['satker', 'nama', 'alamat', 'prov', 'ruang', 'kelas', 'total_kamar', 'terisi_lk', 'terisi_pr', 'total_terisi','kosong_lk', 'kosong_pr', 'total_kosong', 'waiting_list','last_update','lat','lng'])



148

In [5]:
faskes_df = pd.read_csv("Faskes - Rumah Sakit.csv",delimiter=',',encoding='ISO-8859-1')
faskes_df.head()

,Unnamed: 0,prov_id,kode_rs,nama_unit,alamat,nama_prov,jenis_faskes,lat,lng
0,1,12,1277015,RS TNI Tk. IV 01.07.03,Jl. Sudirman No.1 Kelurahan,SUMATERA UTARA,Rumah Sakit,3.596056,98.675812
1,2,12,1277011,RS Umum Daerah Padang Sidempuan,"Jl Dr. F.L. Tobing No. 10,Padangsidimpuan",SUMATERA UTARA,Rumah Sakit,1.419353,99.185211
2,3,12,1276040,RS Tk IV 01.07.02 Binjai,Jl. Bandung No.4 Binjai,SUMATERA UTARA,Rumah Sakit,3.699319,98.507103
3,4,12,1276025,RS Bangkatan PTPN II,Jl. Hasanuddin No.40 Binjai,SUMATERA UTARA,Rumah Sakit,3.600425,98.485710
4,5,12,1275996,RS Umum Sufina Azis,"Jl Karya Baru No.1,Medan",SUMATERA UTARA,Rumah Sakit,3.590356,98.711342


In [6]:
faskes_df.shape

(1222, 9)

In [7]:
for index, row in faskes_df.iterrows():
    satker = str(row['kode_rs'])
    nama_rs = str(row['nama_unit'])
    alamat =  str(row['alamat'])
    prov =  str(row['nama_prov'])
    lat =  row['lat']
    long =  row['lng']

    link = 'http://sirs.yankes.kemkes.go.id/integrasi/data/bed_monitor.php?satker='+satker
    browser.get(link)
    #print(r)
    data = browser.page_source
    url = soup(data,"lxml")
    

    table = url.find('table', attrs={'class':'tbl-responsive table table-striped table-bordered'})
    #print(table)

    if table is not None:
        res = []
        table_rows = table.find_all('tr')

        num_rows = len(table_rows)
        #print(satker+'-'+nama_rs)

        i = 0
        for tr in table_rows:
            _satker = satker
            _nama = nama_rs
            _alamat = alamat
            _prov = prov
            _lat = lat
            _long = long
            _ruang = '-'
            _kelas = '-'
            _total_kamar = '-'
            _terisi_lk = '-'
            _terisi_pr = '-'
            _total_terisi = '-'
            _kosong_lk = '-'
            _kosong_pr = '-'
            _total_kosong = '-'
            _waiting_list = '-'
            _last_update = '-'

            if i>1 and i<(num_rows-1):
                
                td = tr.find_all('td')
                #print(td)
                row = [tr.text.strip() for tr in td if tr.text.strip()]
                #print(row)
                #print(str(i)+'-'+str(len(row)))
                
                #if row:
                if len(row)==12:
                    #print(row)
                    if "Isolasi" in str(row[1]) or "Isolasi" in str(row[2]):
                        #print(satker+'-'+str(len(row)))
                        #print(row)
                        _ruang = row[1]
                        _kelas = row[2]
                        _total_kamar = row[3] 
                        _terisi_lk = row[4] 
                        _terisi_pr = row[5]
                        _total_terisi = row[6]
                        _kosong_lk = row[7]
                        _kosong_pr = row[8]
                        _total_kosong = row[9]
                        _waiting_list = row[10]
                        _last_update = row[11]
                        
                        csvWriter.writerow([_satker, _nama, _alamat, _prov, _ruang, _kelas, _total_kamar, _terisi_lk, _terisi_pr, _total_terisi,  _kosong_lk, _kosong_pr, _total_kosong, _waiting_list, _last_update, _lat, _long])
            
                elif len(row)==11:
                    #print(row)
                    if "Isolasi" in str(row[1]) :
                        #print(satker+'-'+str(len(row)))
                        #print(row[0])
                        _ruang = '-'
                        _kelas = row[1]
                        _total_kamar = row[2] 
                        _terisi_lk = row[3] 
                        _terisi_pr = row[4]
                        _total_terisi = row[5]
                        _kosong_lk = row[6]
                        _kosong_pr = row[7]
                        _total_kosong = row[8]
                        _waiting_list = row[9]
                        _last_update = row[10]
                                  
                        csvWriter.writerow([_satker, _nama, _alamat, _prov, _ruang, _kelas, _total_kamar, _terisi_lk, _terisi_pr, _total_terisi,  _kosong_lk, _kosong_pr, _total_kosong, _waiting_list, _last_update, _lat, _long])
            i = i +1


In [8]:
csvFile.close()

In [9]:
dataset = pd.read_csv(filename)
dataset.shape

(197, 17)

In [10]:
dataset.head(100)

,satker,nama,alamat,prov,ruang,kelas,total_kamar,terisi_lk,terisi_pr,total_terisi,kosong_lk,kosong_pr,total_kosong,waiting_list,last_update,lat,lng
0,1275655,RSUP H. Adam Malik,"Jl Bunga Lau No. 17, Medan",SUMATERA UTARA,Isolasi Air Borne,Kelas 1,11,4,6,10,-,-,1,NaN,27/Mar/2020 09:19:00,3.518900,98.610298
1,1275655,RSUP H. Adam Malik,"Jl Bunga Lau No. 17, Medan",SUMATERA UTARA,Isolasi TB MDR,Kelas 3,10,1,2,3,-,-,7,NaN,27/Mar/2020 09:19:00,3.518900,98.610298
2,1275035,RS Tk. II Putri Hijau Medan,"Jl Putri Hijau No. 17, Medan",SUMATERA UTARA,Isolasi Air Borne,Isolasi,4,2,0,2,0,2,2,NaN,17/Jan/2019 05:41:50,3.595758,98.675636
3,1273011,RS Umum Daerah Dr. Djasamen Saragih,"Jl Sutomo No. 230, Pematangsiantar",SUMATERA UTARA,ICU,Isolasi,5,0,0,0,2,3,5,NaN,27/Mar/2020 04:46:49,2.958633,99.069626
4,1271016,RS Umum Daerah Dr. Fl Tobing Sibolga,Jl Dr. F.L. Tobing No. 35 Sibolga,SUMATERA UTARA,ICU,Isolasi,1,0,0,0,1,1,2,NaN,01/Mar/2019 06:34:39,1.700822,98.822945
5,1201016,RS Umum Daerah Gunungsitoli,Jl dr. Ciptomangunkusumo No 15 Gunungsitoli,SUMATERA UTARA,Isolasi Anak,Rawat Khusus,10,3,4,7,1,2,3,NaN,08/Mar/2019 05:35:45,1.273200,97.622963
6,1372011,RS Umum Daerah Solok,"Jl. Simpang Rumbio,Solok",SUMATERA BARAT,Paru,Isolasi,2,0,0,0,1,1,2,NaN,29/Nov/2018 07:49:57,-1.476997,101.046860
7,1371444,RS Umum Daerah dr. Rasidin Padang,"Jl. Air Paku Sei.Sapih Kec.Kuranji,Padang",SUMATERA BARAT,Umum,Isolasi,0,0,0,0,-,-,0,NaN,25/Feb/2019 07:14:27,0.904914,100.380157
8,1309011,RS Umum Daerah Lubuk Sikaping,"Jl. Jen Sudirman No.33,Lubuk Sikaping",SUMATERA BARAT,Luka Bakar,Isolasi,8,0,0,0,-,-,8,NaN,26/Mar/2020 03:21:50,-0.135597,100.168434
9,1309011,RS Umum Daerah Lubuk Sikaping,"Jl. Jen Sudirman No.33,Lubuk Sikaping",SUMATERA BARAT,Isolasi Anak,Isolasi,2,0,0,0,-,-,2,NaN,26/Mar/2020 03:21:36,-0.135597,100.168434


In [11]:
import pandas_profiling
dataset.profile_report()